# Week 2 - Challenge
## LangChain Practice

### Advanced Customer Service Agent with LangChain

> Deliverables

>> 1. Implementation using LangChain Expression Language (LCEL) to build the agent.
>> 2. Pydantic models for QueryAnalysis and ConversationSummary fully integrated into the chain.
>> 3. Usage examples for each suggested test query, demonstrating the end-to-end functionality.
>> 4. Results analysis. Include a screenshot or public link to a LangSmith trace for one of the complex queries to demonstrate successful tracing.

#### Setup & API Configuration

In [ ]:
# Install packages

!pip install openai
!pip install python-dotenv
!pip install -qU \
  langchain-core \
  langchain-openai \
  langchain-community \
  langsmith

In [ ]:
# Import libraries

import os
from getpass import getpass
import openai
from langchain_openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, PromptTemplate
from langchain.schema.runnable import RunnableLambda#, RunnablePassthrough
from pydantic import BaseModel, Field
from typing import Literal, Optional, List
import json, datetime

In [21]:
# Load environment variables from .env file

# OPENAI_API_KEY

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
or getpass("Enter OpenAI API Key: ")

# LANGCHAIN_API_KEY

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") \
or getpass("Enter LangSmith API Key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "Advanced-Customer-Agent"

client = openai.OpenAI()

# Define the OpenAI model to use

openai_model = "gpt-4o-mini"

# Initialize the OpenAI chat model

llm = ChatOpenAI(model=openai_model,temperature=0)

In [5]:
test_queries = [
    "Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago"
    ,"Urgent-Negative: This is an emergency! My order #TEC-2024-001 never arrived and I need that laptop for work tomorrow!"
    ,"Satisfied-Positive: Thank you so much for the excellent service with my previous purchase. I want to buy gaming headphones"
    ,"Frustrated-Technical: I can't configure the router I bought last week, I've tried everything and it doesn't work"
    ,"Formal-Billing: Good morning, I need the receipt for my purchase from December 15th, order #TEC-2023-089"
    ,"Warranty-Query: I bought a tablet 8 months ago and now it won't turn on, how do I use the warranty?"
]

#### Pydantic Objects

In [6]:
# Define the data model for the extracted information

class ExtractedEntities(BaseModel):
    product_name: Optional[str] = Field(None, description="The specific product mentioned by the user")
    order_number: Optional[str] = Field(None, description="The order number mentioned by the user")
    date_info: Optional[str] = Field(None, description="The date mentioned by the user, use the days of the week, and not 'today,' 'tomorrow' or 'yesterday.' (only if applicable)")

In [7]:
# Define the query analysis model

class QueryAnalysis(BaseModel):
    """Analyzes and classifies a customer query."""
    query_category: Literal["technical_support","billing","returns","product_inquiry","general_information"]
    urgency_level: Literal["low","medium","high"]
    customer_sentiment: Literal["positive","neutral","negative"]
    entities: ExtractedEntities  

In [8]:
# Define the prompt template to analyze the user message and extract relevant information

class ConversationSummary(BaseModel):
    """A structured summary of the customer service interaction."""
    timestamp: str
    customer_id: str = "auto_generated"
    conversation_summary: str = Field(description="A concise, one-sentence summary of the interaction.")
    query_category: str
    customer_sentiment: str
    urgency_level: str
    mentioned_products: List[str]
    extracted_information: dict
    resolution_status: Literal["resolved", "pending", "escalated"]
    actions_taken: List[str] = Field(description="A list of actions the agent took or suggested.")
    follow_up_required: bool

#### 1° Component - Query Analysis & Classification

Analyze the initial User message and extract key information into a structured format.

In [9]:
# Model selected

print(llm.model_name)

gpt-4o-mini


In [10]:
# Sample of the first query

user_message = test_queries[0]
user_message

"Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago"

In [11]:
# Define the system prompt for the chatbot
# This prompt sets the context and behavior of the chatbot

system_prompt = SystemMessagePromptTemplate.from_template(
"""
You are an intelligent assistant that analyzes customer service messages and classifies them with the following criteria:
    1. Analyze the customer message and extract relevant information based on the pydantic model "QueryAnalysis".
    2. Be accurate in your classification.
    3. Respond ONLY with valid JSON structure
""")

print(system_prompt.format().content)


You are an intelligent assistant that analyzes customer service messages and classifies them with the following criteria:
    1. Analyze the customer message and extract relevant information based on the pydantic model "QueryAnalysis".
    2. Be accurate in your classification.
    3. Respond ONLY with valid JSON structure



In [12]:
# Create a specific prompt for the user message analysis

user_prompt = HumanMessagePromptTemplate.from_template(
"""
Analyze the customer message: {user_message}
""", 
    input_variables=["user_message"],
)

print(user_prompt.format(user_message=user_message).content)


Analyze the customer message: Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago



In [20]:
# Create the prompt template for analysis and classification

analysis_classification_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    user_prompt
])

# Create the chain
# This chain will perform the steps: prompt formatting > llm generation > get output in JSON format.

analysis_classification_chain = (
    analysis_classification_prompt
    | llm.with_structured_output(QueryAnalysis)
)

# Test the chain

analysis_classification_chain_test = analysis_classification_chain.invoke({"user_message": user_message})

print(analysis_classification_chain_test.model_dump_json(indent=2))

{
  "query_category": "product_inquiry",
  "urgency_level": "medium",
  "customer_sentiment": "neutral",
  "entities": {
    "product_name": "iPhone 15",
    "order_number": null,
    "date_info": null
  }
}


#### 2° Component - Dynamic Response Generation

Generate a context-aware, personalized response.

In [197]:
print(test_queries[0])

Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago


In [222]:
system_category_prompts = {
    "technical_support": PromptTemplate.from_template(
        """
        You are a helpful and friendly assistant for TechStore Plus that should be empathetic, patient, and ask for relevant troubleshooting steps.
        User message: {user_message}
        Customer sentiment: {customer_sentiment}
        Extracted info: {entities}
        """),

    "billing": PromptTemplate.from_template(
        """
        You are a helpful and friendly assistant for TechStore Plus that should be clear and polite, asking for details like invoice number or payment method.
        User message: {user_message}
        Customer sentiment: {customer_sentiment}
        Extracted info: {entities}
        """),

    "returns": PromptTemplate.from_template(
        """
        You are a helpful and friendly assistant for TechStore Plus that should be understanding and explain the return process, asking for order number if needed.
        User message: {user_message}
        Customer sentiment: {customer_sentiment}
        Extracted info: {entities}
        """),

    "product_inquiry": PromptTemplate.from_template(
        """
        You are a helpful and friendly assistant for TechStore Plus that should be helpful and informative, providing accurate product details.
        User message: {user_message}
        Customer sentiment: {customer_sentiment}
        Extracted info: {entities}
        """),

    "product_inquiry": PromptTemplate.from_template(
        """
        You are a helpful and friendly assistant for TechStore Plus that should be helpful and informative, providing accurate product details.
        User message: {user_message}
        Customer sentiment: {customer_sentiment}
        Extracted info: {entities}
        """),

    "general_information": PromptTemplate.from_template(
        """
        You are a helpful and friendly assistant for TechStore Plus that should be courteous and provide concise, relevant information or guidance.
        User message: {user_message}
        Customer sentiment: {customer_sentiment}
        Extracted info: {entities}
        """)
}

#### 3° Component - Conversation Summarization & Persistance

Generate a structured summary of the entire interaction, ready for logging.

#### 4° LangSmith Integration

Observability and monitoring

In [39]:
print("LangSmith -- Tracing Projects -- Advanced-Customer-Agent: \n\n" \
">> Test chain on customer message: \n" \
"Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago \n\n"
"https://smith.langchain.com/o/ca30c0fa-d39b-4e14-892a-aa6b8b3f7caa/projects/p/7b350e83-86d5-43b2-b195-a83cb9d98184?timeModel=%7B%22duration%22%3A%227d%22%7D")

LangSmith -- Tracing Projects -- Advanced-Customer-Agent: 

>> Test chain on customer message: 
Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago 

https://smith.langchain.com/o/ca30c0fa-d39b-4e14-892a-aa6b8b3f7caa/projects/p/7b350e83-86d5-43b2-b195-a83cb9d98184?timeModel=%7B%22duration%22%3A%227d%22%7D
